In [6]:
import telebot
from telebot import types

bot = telebot.TeleBot('7648430868:AAE0L0yaCWHJHSXIhZcEGnglhZOI0mvcgps')

@bot.message_handler(commands=['start'])
def start(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton('👋 Поздороваться')
    markup.add(btn1)
    bot.send_message(message.chat.id, '👋 Привет! Я твой бот-помощник, давай начнем!', reply_markup=markup)

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    if message.text == '👋 Поздороваться':
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton('Рассчитать суточную норму калорий🍫')
        markup.add(btn1)
        bot.send_message(message.chat.id, ' Выберите, что вас интересует', reply_markup=markup)

    elif message.text == 'Рассчитать суточную норму калорий🍫':
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn_male = types.KeyboardButton('Мужской👨🏻')
        btn_female = types.KeyboardButton('Женский👩🏻')
        markup.add(btn_male, btn_female)
        bot.send_message(message.chat.id, 'Выберите ваш пол, это важно!', reply_markup=markup)

    elif message.text == 'Мужской👨🏻':
        bot.send_message(message.chat.id, 'Ваш пол: Мужской. Введите ваш вес (в кг)')
        bot.register_next_step_handler(message, process_weight, 'Мужской')

    elif message.text == 'Женский👩🏻':
        bot.send_message(message.chat.id, 'Ваш пол: Женский. Введите ваш вес (в кг)')
        bot.register_next_step_handler(message,'Женский')
        
def process_weight(message, gender):
    try:
        weight = float(message.text)
        bot.send_message(message.chat.id, f'Ваш вес: {weight} кг. Введите ваш возраст (в годах)')
        bot.register_next_step_handler(message, lambda msg: process_age(msg, weight, gender))

    except ValueError:
        bot.send_message(message.chat.id, 'Пожалуйста, введите корректный вес в килограммах.')
        bot.register_next_step_handler(message, lambda msg: process_weight(msg, gender))

def calculate_bmr(weight, height, age, gender):
    if gender == 'мужчина':
        return 10 * weight + 6.25 * height - 5 * age + 5
    else:
        return 10 * weight + 6.25 * height - 5 * age - 161

def process_age(message, weight, gender):
    try:
        age = int(message.text)
        bot.send_message(message.chat.id, f'Ваш возраст: {age} лет. Введите ваш рост (в см)')
        bot.register_next_step_handler(message, lambda msg: process_height(msg, weight, gender, age))
    except ValueError:
        bot.send_message(message.chat.id, 'Пожалуйста, введите корректный возраст в годах.')
        bot.register_next_step_handler(message, lambda msg: process_age(msg, weight, gender))

def process_height(message, weight, gender, age):
    try:
        height = float(message.text)
        bot.send_message(message.chat.id, f'Ваш рост: {height} см')

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        activities = ['минимальный', 'низкий', 'умеренный', 'высокий', 'экстремальный', 'не знаю']
        for activity in activities:
            markup.add(types.KeyboardButton(activity))
        bot.send_message(message.chat.id, 'Выберите ваш уровень физической активности', reply_markup=markup)
        bot.register_next_step_handler(message, lambda msg: process_activity(msg, weight, height, age, gender))
    except ValueError:
        bot.send_message(message.chat.id, 'Пожалуйста, введите корректный рост в сантиметрах.')
        bot.register_next_step_handler(message, lambda msg: process_height(msg, weight, gender, age))

def process_activity(message, weight, height, age, gender):
    activity_level = message.text
    activity_factors = {
        'минимальный': 1.2,
        'низкий': 1.375,
        'умеренный': 1.55,
        'высокий': 1.7,
        'экстремальный': 1.9}

    if activity_level in activity_factors:
        bmr = calculate_bmr(weight, height, age, gender)
        maintenance_calories = bmr * activity_factors[activity_level]
        bot.send_message(message.chat.id, f'Ваш BMR: {bmr:.2f} калорий⚡️.\nДля поддержания вашего веса вам необходимо около {maintenance_calories:.2f} калорий в день🍗.')
    else:
        bot.send_message(message.chat.id, 
"минимальный (сидячая работа, отсутствие физических нагрузок); \n"
"низкий (тренировки не менее 20 мин 1-3 раза в неделю); \n"
"умеренный (тренировки 30-60 мин 3-4 раза в неделю);\n "
"высокий (тренировки 30-60 мин 5-7 раз в неделю; тяжелая физическая работа);\n "
"экстремальный (несколько интенсивных тренировок в день 6-7 раз в неделю; очень трудоемкая работа).\n")
        bot.register_next_step_handler(message, lambda msg: process_activity(msg, weight, height, age, gender))

bot.polling(none_stop=True)